In [6]:
# Re-run the gif creation process due to earlier interruption
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import imageio.v2 as imageio
import io

# Re-load the CSV file
fulldatasetpath = '/Volumes/Data_Drive/datasets/VIDIMU'
file_path = os.path.join(fulldatasetpath,'dataset','videoandimusyncrop/S40/S40_A01_T01.csv')
df = pd.read_csv(file_path)
df = df.iloc[:,1:]
print(df.columns[:3])
joint_names_updated = [df.columns[j][:-2] for j in range(1, 103, 3)]

Index(['pelvis_x', 'pelvis_y', 'pelvis_z'], dtype='object')


In [7]:
print(joint_names_updated)

['pelvis', 'left_hip', 'right_hip', 'torso', 'left_knee', 'right_knee', 'neck', 'left_ankle', 'right_ankle', 'left_big_toe', 'right_big_toe', 'left_small_toe', 'right_small_toe', 'left_heel', 'right_heel', 'nose', 'left_eye', 'right_eye', 'left_ear', 'right_ear', 'left_shoulder', 'right_shoulder', 'left_elbow', 'right_elbow', 'left_wrist', 'right_wrist', 'left_pinky_knuckle', 'right_pinky_knuckle', 'left_middle_tip', 'right_middle_tip', 'left_index_knuckle', 'right_index_knuckle', 'left_thumb_tip', ' right_thumb_tip']


## Normalize

In [8]:
import numpy as np

def normalize_joint_positions(joint_data, root_joint='pelvis', epsilon=1e-6):
    """
    Normalize joint positions using a root alignment and local coordinate transformation.

    Parameters:
    - joint_data: DataFrame containing the joint positions (columns: 'joint_x', 'joint_y', 'joint_z')
    - root_joint: The joint to be used as the reference root
    - epsilon: Small value to avoid division by zero

    Returns:
    - normalized_data: DataFrame with normalized joint positions
    """
    # Center data around the root joint
    root_positions = joint_data[[f'{root_joint}_x', f'{root_joint}_y', f'{root_joint}_z']].values
    normalized_data = joint_data.copy()

    # Subtract the root joint position from all other joints to normalize
    for joint in joint_names_updated:
        normalized_data[[f'{joint}_x']] -= root_positions[:, 0][:, None]
        normalized_data[[f'{joint}_y']] -= root_positions[:, 1][:, None]
        normalized_data[[f'{joint}_z']] -= root_positions[:, 2][:, None]

    # Optionally, rotate the joints to align the pelvis-to-torso axis with a standard vertical axis
    # Implement rotation logic here

    return normalized_data

# Example use
normalized_df = normalize_joint_positions(df, root_joint='pelvis')


In [9]:
def align_pelvis_torso_to_z_axis(joint_data, joint_names, pelvis_joint='pelvis', torso_joint='torso', epsilon=1e-6):
    """
    Rotates the skeleton such that the pelvis-to-torso vector aligns with the z-axis in the first frame.
    
    Parameters:
    - joint_data: DataFrame containing the joint positions (columns: 'joint_x', 'joint_y', 'joint_z')
    - joint_names: List of joint names
    - pelvis_joint: The name of the pelvis joint
    - torso_joint: The name of the torso joint
    - epsilon: Small value to avoid division by zero
    
    Returns:
    - rotated_data: DataFrame with rotated joint positions
    """
    # Get the initial pelvis-to-torso vector in the first frame
    pelvis_position = joint_data[[f'{pelvis_joint}_x', f'{pelvis_joint}_y', f'{pelvis_joint}_z']].values[0]
    torso_position = joint_data[[f'{torso_joint}_x', f'{torso_joint}_y', f'{torso_joint}_z']].values[0]
    pelvis_to_torso = torso_position - pelvis_position
    
    # Normalize the pelvis-to-torso vector
    pelvis_to_torso /= (np.linalg.norm(pelvis_to_torso) + epsilon)

    # Desired direction (z-axis)
    z_axis = np.array([0.0, 0.0, 1.0])

    # Compute the rotation axis (cross product of pelvis-to-torso and z-axis)
    rotation_axis = np.cross(pelvis_to_torso, z_axis)
    rotation_axis_norm = np.linalg.norm(rotation_axis)
    
    if rotation_axis_norm < epsilon:
        # If the rotation axis norm is too small, the vectors are already aligned
        rotation_matrix = np.eye(3)
    else:
        rotation_axis /= rotation_axis_norm  # Normalize the rotation axis

        # Compute the angle between the pelvis-to-torso vector and the z-axis
        angle = np.arccos(np.clip(np.dot(pelvis_to_torso, z_axis), -1.0, 1.0))

        # Rodrigues' rotation formula
        K = np.array([
            [0, -rotation_axis[2], rotation_axis[1]],
            [rotation_axis[2], 0, -rotation_axis[0]],
            [-rotation_axis[1], rotation_axis[0], 0]
        ])
        I = np.eye(3)
        rotation_matrix = I + np.sin(angle) * K + (1 - np.cos(angle)) * np.dot(K, K)

    # Apply the rotation matrix to all joints
    rotated_data = joint_data.copy()
    for joint in joint_names:
        joint_coords = joint_data[[f'{joint}_x', f'{joint}_y', f'{joint}_z']].values
        rotated_coords = np.dot(joint_coords, rotation_matrix.T)
        rotated_data[f'{joint}_x'], rotated_data[f'{joint}_y'], rotated_data[f'{joint}_z'] = rotated_coords.T

    return rotated_data

# Example use
normalized_df = align_pelvis_torso_to_z_axis(normalized_df, joint_names_updated, pelvis_joint='pelvis', torso_joint='torso')


## Plot

In [10]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import io
import imageio.v2 as imageio

# Extract the XYZ coordinates for each joint
joint_coords_updated = {}
for joint in joint_names_updated:
    joint_coords_updated[joint] = normalized_df[[f'{joint}_x', f'{joint}_y', f'{joint}_z']].values

# Get the min and max values for each axis (for consistent plot scaling)
x_min, x_max = normalized_df[[f'{joint}_x' for joint in joint_names_updated]].min().min(), normalized_df[[f'{joint}_x' for joint in joint_names_updated]].max().max()
y_min, y_max = normalized_df[[f'{joint}_y' for joint in joint_names_updated]].min().min(), normalized_df[[f'{joint}_y' for joint in joint_names_updated]].max().max()
z_min, z_max = normalized_df[[f'{joint}_z' for joint in joint_names_updated]].min().min(), normalized_df[[f'{joint}_z' for joint in joint_names_updated]].max().max()

# Function to plot the skeleton with a fixed view and grid
def plot_skeleton_frame_with_axes(ax, frame_idx):
    joints = {joint: joint_coords_updated[joint][frame_idx] for joint in joint_names_updated}

    # Updated skeleton connections based on the available joints
    connections_updated = [
        ('pelvis', 'left_hip'), ('pelvis', 'right_hip'),
        ('left_hip', 'left_knee'), ('right_hip', 'right_knee'),
        ('left_knee', 'left_ankle'), ('right_knee', 'right_ankle'),
        ('left_ankle', 'left_heel'), ('right_ankle', 'right_heel'),
        ('left_heel', 'left_big_toe'), ('right_heel', 'right_big_toe'),
        ('left_heel', 'left_small_toe'), ('right_heel', 'right_small_toe'),
        ('pelvis', 'torso'), ('torso', 'neck'), ('neck', 'nose'),
        ('neck', 'left_shoulder'), ('neck', 'right_shoulder'),
        ('left_shoulder', 'left_elbow'), ('right_shoulder', 'right_elbow'),
        ('left_elbow', 'left_wrist'), ('right_elbow', 'right_wrist'),
        ('left_wrist', 'left_pinky_knuckle'), ('right_wrist', 'right_pinky_knuckle'),
        ('left_wrist', 'left_index_knuckle'), ('right_wrist', 'right_index_knuckle'),
        ('left_wrist', 'left_thumb_tip'), ('right_wrist', 'right_thumb_tip')
    ]

    # Extract joint positions for plotting
    xs, ys, zs = [], [], []
    for joint in joint_names_updated:
        if joint in joints:  # Ensure the joint is available in the current frame
            x, y, z = joints[joint]
            xs.append(x)
            ys.append(y)
            zs.append(z)

    # Plot joints and connections
    ax.scatter(xs, ys, zs, color='blue')
    for conn in connections_updated:
        if conn[0] in joints and conn[1] in joints:  # Ensure both joints in the connection exist
            x_vals = [joints[conn[0]][0], joints[conn[1]][0]]
            y_vals = [joints[conn[0]][1], joints[conn[1]][1]]
            z_vals = [joints[conn[0]][2], joints[conn[1]][2]]
            ax.plot(x_vals, y_vals, z_vals, color='red')

    # Set the axis limits dynamically based on the entire dataset
    ax.set_xlim([x_min, x_max])
    ax.set_ylim([y_min, y_max])
    ax.set_zlim([z_min, z_max])

    # Show grid and axes
    ax.grid(True)
    ax.set_xlabel('X Axis')
    ax.set_ylabel('Y Axis')
    ax.set_zlabel('Z Axis')

# Create gif with dynamic movement and fixed view
gif_images_dynamic = []
for frame in range(0, normalized_df.shape[0], 10):  # Every 10th frame for speed
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')

    # Set a predefined view
    # ax.view_init(elev=30, azim=-90)  # Change these values to get the desired initial perspective

    # Plot the skeleton for this frame
    plot_skeleton_frame_with_axes(ax, frame)

    # Save figure to a buffer instead of a file
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    gif_images_dynamic.append(imageio.imread(buf))
    buf.close()
    plt.close(fig)  # Close the figure after saving to buffer

# Save the gif without temporary files
imageio.mimsave('skeleton_3d_normalized5.gif', gif_images_dynamic, fps=10)